In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkContext,SQLContext
sc =SparkContext()
sc.addPyFile('C:/Users/ramya/Anaconda3/Lib/site-packages/pyspark/jars/graphframes-0.8.0-spark2.4-s_2.11.jar')
spark = SparkSession(sc)

from pyspark.sql.types import *
from graphframes import *

In [2]:
bikeStations = spark.read.option("header","true").option("inferschema", "true")\
  .csv("C:/Users/ramya/Desktop/Santa Clara University/Q3/Pyspark/station.csv")
tripData = spark.read.option("header","true").option("inferschema", "true")\
  .csv("C:/Users/ramya/Desktop/Santa Clara University/Q3/Pyspark/trip.csv")

In [3]:
type(tripData)

pyspark.sql.dataframe.DataFrame

In [4]:
bikeStations.show(3)

+---+--------------------+---------+-------------------+----------+--------+-----------------+
| id|                name|      lat|               long|dock_count|    city|installation_date|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
|  2|San Jose Diridon ...|37.329732|-121.90178200000001|        27|San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|37.330698|        -121.888979|        15|San Jose|         8/5/2013|
|  4|Santa Clara at Al...|37.333988|        -121.894902|        11|San Jose|         8/6/2013|
+---+--------------------+---------+-------------------+----------+--------+-----------------+
only showing top 3 rows



In [5]:
tripData.show(3)

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|      70|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...|              27|8/29/2013 10:17|Mountain View Cit...|            27|     48|       Subscriber|   97214|
+----+--------+---------------+--------------------+------

In [6]:
bikeStations = bikeStations.withColumn("id", bikeStations["id"].cast(IntegerType()))
tripEdges = tripData\
  .withColumnRenamed("start_station_id", "src")\
  .withColumnRenamed("end_station_id", "dst")

stationGraph = GraphFrame(bikeStations, tripEdges)
stationGraph.cache()

GraphFrame(v:[id: int, name: string ... 5 more fields], e:[src: int, dst: int ... 9 more fields])

In [7]:
bikeStations.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



In [10]:
tripEdges.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- src: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- dst: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [7]:
str(stationGraph.vertices.count())
stationGraph.edges.count()

669959

In [43]:
(tripData.count())

669959

In [44]:
stationGraph.edges.count()

354152

In [8]:
from pyspark.sql.functions import desc
stationGraph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show(10,truncate=False)

+---+---+-----+
|src|dst|count|
+---+---+-----+
|69 |65 |6216 |
|50 |60 |6164 |
|65 |70 |5041 |
|61 |50 |4839 |
|50 |61 |4357 |
|60 |74 |4269 |
|51 |70 |3967 |
|74 |61 |3903 |
|64 |77 |3627 |
|70 |50 |3622 |
+---+---+-----+
only showing top 10 rows



In [9]:
stationGraph.edges\
  .where("src = 'Townsend at 7th' OR dst = 'Townsend at 7th'")\
  .groupBy("src", "dst").count()\
  .orderBy(desc("count"))\
  .show(10,truncate=False)

+---+---+-----+
|src|dst|count|
+---+---+-----+
+---+---+-----+



In [5]:
stationGraph.find("(a)-[]->(b)").where("a.id!=b.id").count()

645978

In [65]:
inDeg = stationGraph.inDegrees
inDeg.orderBy(desc("inDegree")).show(5, False)

+---+--------+
|id |inDegree|
+---+--------+
|70 |31654   |
|60 |17628   |
|50 |17353   |
|77 |15524   |
|69 |15409   |
+---+--------+
only showing top 5 rows



In [66]:
outDeg = stationGraph.outDegrees
outDeg.orderBy(desc("outDegree")).show(5, False)

+---+---------+
|id |outDegree|
+---+---------+
|70 |25239    |
|50 |17561    |
|60 |15187    |
|69 |14563    |
|77 |13950    |
+---+---------+
only showing top 5 rows



In [74]:
degreeRatio = inDeg.join(outDeg, "id")\
  .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio")
degreeRatio.orderBy(desc("degreeRatio")).show(10, False)
degreeRatio.orderBy("degreeRatio").show(10, False)

+---+------------------+
|id |degreeRatio       |
+---+------------------+
|24 |1.8316831683168318|
|11 |1.5656759348034517|
|26 |1.368421052631579 |
|70 |1.2541701335235151|
|25 |1.2026143790849673|
|46 |1.2019877228880445|
|35 |1.1901408450704225|
|9  |1.177660510114336 |
|60 |1.1607295713439125|
|23 |1.1542857142857144|
+---+------------------+
only showing top 10 rows

+---+------------------+
|id |degreeRatio       |
+---+------------------+
|73 |0.6134626354820308|
|62 |0.6197822141560798|
|10 |0.6312189054726368|
|22 |0.6800976800976801|
|58 |0.7292511988196237|
|71 |0.7312341441575885|
|56 |0.7924323017408124|
|37 |0.8155452436194895|
|82 |0.8323855133193655|
|59 |0.849609375       |
+---+------------------+
only showing top 10 rows



In [112]:
stationGraph.bfs(fromExpr="id = 83",toExpr="id = 26",maxPathLength=2).select("e0").show()

+--------------------+
|                  e0|
+--------------------+
|[192526, 588, 2/2...|
|[192527, 530, 2/2...|
|[833572, 3268, 7/...|
|[833571, 3306, 7/...|
+--------------------+



In [88]:
tripEdges.show(3)

+----+--------+---------------+--------------------+---+---------------+--------------------+---+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|src|       end_date|    end_station_name|dst|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+---+---------------+--------------------+---+-------+-----------------+--------+
|4576|      63|8/29/2013 14:13|South Van Ness at...| 66|8/29/2013 14:14|South Van Ness at...| 66|    520|       Subscriber|   94127|
|4607|      70|8/29/2013 14:42|  San Jose City Hall| 10|8/29/2013 14:43|  San Jose City Hall| 10|    661|       Subscriber|   95138|
|4130|      71|8/29/2013 10:16|Mountain View Cit...| 27|8/29/2013 10:17|Mountain View Cit...| 27|     48|       Subscriber|   97214|
+----+--------+---------------+--------------------+---+---------------+--------------------+---+-------+-----------------+--------+
only showing top 3 rows



In [102]:
from pyspark.sql.functions import countDistinct
tripEdges.groupby("src").agg(countDistinct("end_station_name").alias("end_s")).\
sort('end_s', ascending=True).show(5,False)

+---+-----+
|src|end_s|
+---+-----+
|83 |8    |
|30 |8    |
|26 |10   |
|21 |11   |
|23 |11   |
+---+-----+
only showing top 5 rows



In [113]:
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")

In [12]:
stationGraph.triangleCount().orderBy(desc("count")).show(5)

+-----+---+--------------------+------------------+-------------------+----------+-------------+-----------------+
|count| id|                name|               lat|               long|dock_count|         city|installation_date|
+-----+---+--------------------+------------------+-------------------+----------+-------------+-----------------+
|  603| 70|San Francisco Cal...|         37.776617|-122.39526000000001|        19|San Francisco|        8/23/2013|
|  567| 72|Civic Center BART...|         37.781039|        -122.411748|        23|San Francisco|        8/23/2013|
|  566| 39|  Powell Street BART|37.783871000000005|        -122.408433|        19|San Francisco|        8/25/2013|
|  565| 57|       5th at Howard|37.781752000000004|-122.40512700000001|        15|San Francisco|        8/21/2013|
|  565| 76|       Market at 4th|         37.786305|-122.40496599999999|        19|San Francisco|        8/25/2013|
+-----+---+--------------------+------------------+-------------------+---------

In [12]:
stationGraph.vertices.select("long").show(3)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:65281)
Traceback (most recent call last):
  File "C:\Users\ramya\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1145, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ramya\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "C:\Users\ramya\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1149, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ramya\Anaconda3\lib\site-package

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:65281)

In [28]:
from pyspark.sql.functions import *
stationGraph.vertices.agg(max(col("long")).alias("east"),min(col("long")).alias("west"),\
                          max(col("lat")).alias("north"),min(col("lat")).alias("south")).show()

+-----------+-----------+--------+---------+
|       east|       west|   north|    south|
+-----------+-----------+--------+---------+
|-121.877349|-122.418954|37.80477|37.329732|
+-----------+-----------+--------+---------+

